<a href="https://colab.research.google.com/github/Jin0331/TA/blob/master/Bigdata_2020/Bigdata_Lab_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **빅데이터 개론 Lab 12 - 의사 결정 트리(Decision Tree)**

참고자료 : https://www.notion.so/TA-2689a38b5289413a82671d3956fea103

- - -




### **<의사 결정 트리(Decision Tree>**

* 기계학습 중 하나로 특정 항목에 대한 의사 결정 규칙 (Decision rule)을 나무 형태로 분류해 나가는 분석 기법

* 예를 들어, 타이타닉 호 탑승자의 성별, 나이, 자녀의 수를 이용해서 생존 확률을 아래와 같이 구분해 나가는 것

<img src="https://i0.wp.com/www.dodomira.com/wp-content/uploads/2016/04/CART_tree_titanic_survivors_KOR.png?w=350" width="600" height="450">

* 가장 큰 장점은 분석 과정이 직관적이고 이해하기 쉬움

* 인공신경망 분석의 경우 결과에 대한 설명을 이해하기 어려운 대표적인 블랙박스 모델인 반면, 의사결정나무 기법은 분석 과정을 실제로 눈으로 관측할 수 있기 때문에 대표적인 ``화이트박스 모델``.

* 수치형(Numeric, Integer)/범주형(Factor) 변수를 모두 사용할 수 있다는 점, 계산 비용이 낮아 대규모의 데이터 셋에서도 비교적 빠르게 연산이 가능하다는 점이 장점.

* 분석 방법에는 통계학에 기반한 (카이스퀘어, T검정, F검정 등을 사용한) CART 및 CHAID 알고리즘이나, 기계학습 계열인(엔트로피, 정보 이득 등을 사용한) ID3, C4.5, C5.0 등의 알고리즘이 존재

### **R의 의사결정나무 분석 패키지 비교 (Packages for Decision Tree in R)**

* tree, rpart, party

* 각각의 패키지는 의사결정나무를 만들 때 가지치기(pruning)를 하는 방법에 차이가 존재

1. tree 패키지는 binary recursive partitioning을,  rpart 패키지는 CART(classification and regression trees) 사용

  * 엔트로피(entropy, tree 패키지), 지니계수(gini index, rpart 패키지)를 기준으로 가지치기를 할 변수를 결정하기 때문에 상대적으로 연산 속도는 빠르지만 과적합화의 위험성 존재. ``따라서 두 패키지를 사용할 경우에는 Pruning 과정을 거쳐서 의사결정트리를 최적화 하는 과정이 필요``

2. party 패키지는 Unbiased recursive partitioning based on permutation tests 방법론을 사용

  * p-test를 거친 Significance를 기준으로 가지치기를 할 변수를 결정하기 때문에 biased 될 위험이 없어 별도로 Pruning할 필요가 없다는 장점 존재. ``하지만, 입력 변수의 레벨이 31개 까지로 제한되어 있다는 단점이 있음``

* **알고리즘 비교**

<img src="https://github.com/Jin0331/TA/blob/master/image/algorithm_dt.png?raw=true" width="600" height="450">

출처 : https://m.blog.naver.com/PostView.nhn?blogId=tjdudwo93&logNo=221041168345&proxyReferer=https:%2F%2Fwww.google.com%2F


In [ ]:
install.packages(c("tidyverse", "data.table", "caret", "e1071", "rpart", "tree", "party"))
library(tidyverse)
library(data.table)
library(caret)
library(rpart)
library(tree)
library(party)

options(repr.plot.width=10, repr.plot.height=10)

#### **A. 예제 1 - rpart 패키지 이용**

---


1. **Heart Disease Data** [http://archive.ics.uci.edu/ml/datasets/heart+Disease]

* 변수 설명

```
Age : age in years
Sex: sex (1 = male; 0 = female) # Factor
ChestPain : (typical angina, atypical angina, non-anginal pain, asymptomatic # Factor
RestBP(혈압) : resting blood pressure
Chol(콜레스테롤 수치) : serum cholestoral in mg/dl
Fbs(혈당) : (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false) # Factor
Restecg(심전도) : (0 = normal, 1 = having ST-T wave abnormality, 2 =  showing probable or definite left ventricular hypertrophy by Estes' criteria) # Factor
MaxHR : maximum heart rate achieved
ExAng(협심증?): exercise induced angina (1 = yes; 0 = no) # Factor
Oldpeak = ST depression induced by exercise relative to rest
Slope: the slope of the peak exercise ST segment(1 = upsloping, 2 = flat, 3 = downsloping) # Factor
Ca: number of major vessels (0-3) colored by flourosopy # Factor
Thal: 3 = normal; 6 = fixed defect; 7 = reversable defect # Factor

# the predicted attribute(반응변수)

AHD : diagnosis of heart disease (angiographic disease status)(0 = < 50% diameter narrowing, 1 =  > 50% diameter narrowing)

# http://archive.ics.uci.edu/ml/datasets/heart+Disease


In [ ]:
heart_df <- fread("https://raw.githubusercontent.com/Jin0331/TA/master/data/heart/Heart.csv") %>% 
  as_tibble()

str(heart_df)

In [ ]:
heart_df %>% str()

In [ ]:
heart_df %>% summary()

* Sex, ChestPain, Fbs, RestECG, ExAng, Slope, Ca, Thal, AHD ---> Factor

In [ ]:
heart_df$ChestPain %>% unique() 

In [ ]:
heart_df$Fbs %>% unique()

In [ ]:
heart_df$RestECG %>% unique()

In [ ]:
heart_df$ExAng %>% unique()

In [ ]:
heart_df$Slope %>% unique()

In [ ]:
heart_df$Ca %>% unique()

In [ ]:
heart_df$Thal %>% unique()

In [ ]:
heart_df$AHD %>% unique()

* mutate를 이용한 데이터 타입 변경(int or chr ---> factor)

In [ ]:
heart_df <- heart_df %>% 
 mutate_at(`.vars` = c("Sex", "ChestPain", "Fbs", "RestECG", "ExAng", "Slope", "Ca", "Thal", "AHD"), `.funs` = as.factor)
heart_df %>% str()

* **train-test split**

In [ ]:
library(caret) 
set.seed(31)

index <- createDataPartition(y = heart_df$AHD, p = 0.7, list = FALSE) 
head(index, 20)

In [ ]:
train <- heart_df[index, ]
test <- heart_df[-index, ]

train %>% show()
test %>% show()

* **train 데이터 및 rpart를 이용한 의사결정트리(Decision Tree) 모델 생성**

![png](https://github.com/Jin0331/TA/blob/master/image/rpart_help.png?raw=true)

In [ ]:
library(rpart)
AHD_detection <- rpart(formula = AHD ~ ., data = train, method = "class")


* plotting

In [ ]:
plot(AHD_detection)
text(AHD_detection)

* 심미적 plotting(?)

In [ ]:
install.packages(c("rattle", "rpart.plot"))

library(rattle)
library(rpart.plot)
library(RColorBrewer)

In [ ]:
fancyRpartPlot(AHD_detection)

A. node의 성질

* 박스 가장 위의 “No”라는 구분자는 해당 node는 “no”(음성)라고 구분될 수 있다는 것을 의미. 같은 원리로 가장 아랫줄의 제일 오른쪽  node(7번)를 보면 "yes". 해당 node에 속하는 관측치(여기서는 환자들)은 양성그룹으로 분류

B. node의 순도

* node의 순도(지니 불순도)는 node의 색상(진하기) 및 두번째 열의 숫자로 확인. 7번 node는 0.04, 0.96으로 표시되어 있음. 이 의미는 이 node에 속하는 관측치 중 4%는 no 96%는 yes에 속한다는 의미. 

C. node가 전체에서 차지하는 비중

* 7번 node로 돌아가서, 가장 아랫쪽의 숫자 25%가 의미하는 것은 해당 node가 전체 데이터 셋에서 차지하는 비중. 전체의 25%의 관측치(환자)가 이 node에 속한다고 볼 수 있음.

```
종합적으로 해석해 보면 7번 node로 분류된 환자는 전체의 25%로,
ChestPain이 'typical''nonanginal''nontypical'이 아니면서, 
Ca가 0이 아닌 환자들로 구성된 이 그룹은 96%의 확률로 심장병 양성에 속한다는 해석을 할 수 있다. 

```




* rpart::printcp을 이용한 가지치기(pruning)

In [ ]:
rpart::printcp(AHD_detection)

In [ ]:
plotcp(AHD_detection)

  - xerror(cross validation error)가 최소가 되는 CP를 선택

In [ ]:
AHD_detection$cptable %>% as_tibble() %>%
  filter(xerror == min(xerror))

min_xerror_cp <- AHD_detection$cptable %>% as_tibble() %>%
  filter(xerror == min(xerror)) %>% pull(CP)

* prune

In [ ]:
AHD_detection_pr <- rpart::prune(AHD_detection, cp = min_xerror_cp)

fancyRpartPlot(AHD_detection_pr)

In [ ]:
# 이전 그래프
fancyRpartPlot(AHD_detection)

* **test를 이용한 예측 및 평가**

In [ ]:
test %>% show()

In [ ]:
predict_value <- predict(AHD_detection_pr, test, type = "class") %>% 
 tibble(predict_value = .)
predict_value %>% show()

In [ ]:
predict_check <- test %>% select(AHD) %>% dplyr::bind_cols(., predict_value) 
predict_check %>% show()

* Confusion Matrix(실제값과 모델에 의한 분류값을 비교하는 테이블)

![png](https://github.com/Jin0331/TA/blob/master/image/confusion_m.png?raw=true)

https://yamalab.tistory.com/50

In [ ]:
cm <- caret::confusionMatrix(predict_value$predict_value, test$AHD)
cm

In [ ]:
draw_confusion_matrix(cm)

- - -

#### **B. 예제 2**

* https://www.kaggle.com/c/titanic/data

**<kaggle의 타이타닉 data>**

  * survived : 생존=1, 죽음=0
  * pclass : 승객 등급. 1등급=1, 2등급=2, 3등급=3
  * sibsp : 함께 탑승한 형제 또는 배우자 수
  * parch : 함께 탑승한 부모 또는 자녀 수
  * ticket : 티켓 번호
  * cabin : 선실 번호
  * embarked : 탑승장소 S=Southhampton, C=Cherbourg, Q=Queenstown

In [ ]:
train <- fread("https://raw.githubusercontent.com/Jin0331/TA/master/data/titanic/train.csv") %>%
 as_tibble()

In [ ]:
str(train)

In [ ]:
train %>% summary()

* 범주형 변수 확인

In [ ]:
train$Survived %>% unique()

In [ ]:
train$Pclass %>% unique()

In [ ]:
train$Sex %>% unique()

In [ ]:
train$Ticket %>% unique()

In [ ]:
train$Embarked %>% unique()

In [ ]:
train <- train %>% 
 select(-PassengerId, -Name, -Cabin, -Ticket) %>% mutate_at(c("Survived","Sex","Embarked", "Pclass"), factor)
summary(train)

* Hmisc::impute을 이용한 NA 값 대체(평균, 중앙값, 특정 숫자)

* https://m.blog.naver.com/PostView.nhn?blogId=tjdudwo93&logNo=221142961499&proxyReferer=https:%2F%2Fwww.google.com%2F

In [ ]:
install.packages("Hmisc")

In [ ]:
library(Hmisc)
train$Age <- impute(train$Age, median)

In [ ]:
train %>% summary()

* **train을 이용한 Decision Tree 모델 생성**

In [ ]:
library(rpart)
Survived_detection <- rpart(formula = Survived ~ ., data = train, method = "class")

In [ ]:
# plotting
fancyRpartPlot(Survived_detection)

* pruning

In [ ]:
printcp(Survived_detection)

In [ ]:
plotcp(Survived_detection)

In [ ]:
min_xerror_cp <- Survived_detection$cptable %>% as_tibble() %>%
  filter(xerror == min(xerror)) %>% pull(CP)
min_xerror_cp

In [ ]:
Survived_detection_pr <- rpart::prune(Survived_detection, cp = 0.017)

fancyRpartPlot(Survived_detection_pr)

In [ ]:
fancyRpartPlot(Survived_detection)

* 생성한 2개의 Decision Tree 모델을 이용하여 kaggle에 제출해보기 ㅎ

In [ ]:
test <- fread("https://raw.githubusercontent.com/Jin0331/TA/master/data/titanic/test.csv", sep = ",") %>% as_tibble()
test %>% summary()

* NA 값 추정(median)

In [ ]:
test$Age <- impute(test$Age, median)
test$Fare <- impute(test$Age, median)
test %>% summary()

* 범주형 변수

In [ ]:
test <- test %>% 
 select(-Name, -Cabin, -Ticket) %>% mutate_at(c("Sex","Embarked", "Pclass"), factor)
summary(test)

* 예측(Survived_detection_pr, Survived_detection 모델)

In [ ]:
# pruning 모델
predict_value <- predict(Survived_detection_pr, test, type = "class") %>% tibble(Survived = .)
submission_pr <- test %>% select(PassengerId) %>% dplyr::bind_cols(., predict_value)

# 기존 모델
predict_value <- predict(Survived_detection, test, type = "class") %>% tibble(Survived = .)
submission <- test %>% select(PassengerId) %>% dplyr::bind_cols(., predict_value)

In [ ]:
# id 900 차이
submission_pr %>% head(20)

In [ ]:
submission %>% head(20)

In [ ]:
 # write
 submission_pr %>% write_csv(path = "submission_pr.csv")
 submission %>% write_csv(path = "submission.csv")

### Confusion Matrix plot code

In [ ]:
#https://stackoverflow.com/questions/23891140/r-how-to-visualize-confusion-matrix-using-the-caret-package
draw_confusion_matrix <- function(cm) {

  total <- sum(cm$table)
  res <- as.numeric(cm$table)

  # Generate color gradients. Palettes come from RColorBrewer.
  greenPalette <- c("#F7FCF5","#E5F5E0","#C7E9C0","#A1D99B","#74C476","#41AB5D","#238B45","#006D2C","#00441B")
  redPalette <- c("#FFF5F0","#FEE0D2","#FCBBA1","#FC9272","#FB6A4A","#EF3B2C","#CB181D","#A50F15","#67000D")
  getColor <- function (greenOrRed = "green", amount = 0) {
    if (amount == 0)
      return("#FFFFFF")
    palette <- greenPalette
    if (greenOrRed == "red")
      palette <- redPalette
    colorRampPalette(palette)(100)[10 + ceiling(90 * amount / total)]
  }

  # set the basic layout
  layout(matrix(c(1,1,2)))
  par(mar=c(2,2,2,2))
  plot(c(100, 345), c(300, 450), type = "n", xlab="", ylab="", xaxt='n', yaxt='n')
  title('CONFUSION MATRIX', cex.main=2)

  # create the matrix 
  classes = colnames(cm$table)
  rect(150, 430, 240, 370, col=getColor("green", res[1]))
  text(195, 435, classes[1], cex=1.2)
  rect(250, 430, 340, 370, col=getColor("red", res[3]))
  text(295, 435, classes[2], cex=1.2)
  text(125, 370, 'Predicted', cex=1.3, srt=90, font=2)
  text(245, 450, 'Actual', cex=1.3, font=2)
  rect(150, 305, 240, 365, col=getColor("red", res[2]))
  rect(250, 305, 340, 365, col=getColor("green", res[4]))
  text(140, 400, classes[1], cex=1.2, srt=90)
  text(140, 335, classes[2], cex=1.2, srt=90)

  # add in the cm results
  text(195, 400, res[1], cex=1.6, font=2, col='white')
  text(195, 335, res[2], cex=1.6, font=2, col='white')
  text(295, 400, res[3], cex=1.6, font=2, col='white')
  text(295, 335, res[4], cex=1.6, font=2, col='white')

  # add in the specifics 
  plot(c(100, 0), c(100, 0), type = "n", xlab="", ylab="", main = "DETAILS", xaxt='n', yaxt='n')
  text(10, 85, names(cm$byClass[1]), cex=1.2, font=2)
  text(10, 70, round(as.numeric(cm$byClass[1]), 3), cex=1.2)
  text(30, 85, names(cm$byClass[2]), cex=1.2, font=2)
  text(30, 70, round(as.numeric(cm$byClass[2]), 3), cex=1.2)
  text(50, 85, names(cm$byClass[5]), cex=1.2, font=2)
  text(50, 70, round(as.numeric(cm$byClass[5]), 3), cex=1.2)
  text(70, 85, names(cm$byClass[6]), cex=1.2, font=2)
  text(70, 70, round(as.numeric(cm$byClass[6]), 3), cex=1.2)
  text(90, 85, names(cm$byClass[7]), cex=1.2, font=2)
  text(90, 70, round(as.numeric(cm$byClass[7]), 3), cex=1.2)

  # add in the accuracy information 
  text(30, 35, names(cm$overall[1]), cex=1.5, font=2)
  text(30, 20, round(as.numeric(cm$overall[1]), 3), cex=1.4)
  text(70, 35, names(cm$overall[2]), cex=1.5, font=2)
  text(70, 20, round(as.numeric(cm$overall[2]), 3), cex=1.4)
}